In [ ]:
# The goal of this analysis is to create our own benchmark score using the same methodology as ENERGY STAR uses.
# For reference, their methodology is detailed here: https://portfoliomanager.energystar.gov/pdf/reference/ENERGY%20STAR%20Score.pdf
# Since scores are tied to building type, we are going to only use Multifamily Housing properties to create our scores
# To create their score, Energy Star uses various building characteristics to predict Source EUI
# We have limited building characteristics to work with in our data but will go through the process and see how it works out

In [1]:
# You only need to install pandas once, so you can skip that step and just import the library if it's already installed
!pip install pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.2 MB/s eta 0:00:00a 0:00:01


In [2]:
# Import the pandas library
import pandas as pd

In [3]:
# Load the data file. Replace the path name with the path to the Excel file on your machine.
# To make it easy, put the Excel file in the same folder as the notebook then you only need the file name
# In this case, I put the Excel file in a subfolder called "Reference Materials", which you can remove if you
# have it in the same folder as the notebook file

file_path = 'Reference Materials/nyc_benchmarking_disclosure_2017_consumption_data.xlsx'

#In our data file, we need to load the data from the Information and Metrics sheet specifically so we designate that here
sheet = 'Information and Metrics'

# Read the specific sheet from the Excel file
df = pd.read_excel(file_path, sheet_name=sheet)

# Let's filter our dataframe and just look at one property type of Multifamily.
df_multifamily = df[df['Primary Property Type - Self Selected'] == 'Multifamily Housing']

df_multifamily

FileNotFoundError: [Errno 2] No such file or directory: 'Reference Materials/nyc_benchmarking_disclosure_2017_consumption_data.xlsx'

In [ ]:
# Looking at our data set, we have just a few options for predictor variables for Source EUI
# I think year built, occupancy, and floor area could all be worth looking at as potential predictors
# We will remove rows where 'Source EUI (kBtu/ft²)' and our predictor candidates are missing using the dropna function
# We will also remove rows where natural gas use and electricity use are na as we might want those later
df_multifamily = df_multifamily.dropna(subset=['Source EUI (kBtu/ft²)'])
df_multifamily = df_multifamily.dropna(subset=['Year Built'])
df_multifamily = df_multifamily.dropna(subset=['Occupancy'])
df_multifamily = df_multifamily.dropna(subset=['Self-Reported Gross Floor Area (ft²)'])
df_multifamily = df_multifamily.dropna(subset=['Natural Gas Use (kBtu)'])
df_multifamily = df_multifamily.dropna(subset=['Electricity Use - Grid Purchase (kBtu)'])

# Let's further filter the dataframe for rows where "Number of Buildings" is equal to 1
df_multifamily = df_multifamily[df_multifamily['Number of Buildings'] == 1]

# Filter out rows where 'Occupancy' is 0
df_multifamily = df_multifamily[df_multifamily['Occupancy'] != 0]

print("Number of remaining rows:",len(df_multifamily))

In [ ]:
#Now let's visualize some of the columns we might want to work with to examine the data more closely
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram of the 'Year Built'
plt.hist(df_multifamily['Year Built'].dropna(), bins=50)
plt.title('Distribution of Year Built for Multifamily Properties')
plt.xlabel('Year Built')
plt.ylabel('Number of Properties')
plt.show()

# Histogram of Floor Area (ft2)
plt.hist(df_multifamily['Self-Reported Gross Floor Area (ft²)'].dropna(), bins=100)
plt.title('Distribution of Floor Area for Multifamily Properties')
plt.xlabel('Floor Area (ft2)')
plt.ylabel('Number of Properties')
plt.show()

# Histogram of Occupancy (%)
plt.hist(df_multifamily['Occupancy'].dropna(), bins=20)
plt.title('Distribution of Occupancy for Multifamily Properties')
plt.xlabel('Percent Occupancy')
plt.ylabel('Number of Properties')
plt.show()


In [ ]:
# The charts above show some outlier values but make it hard to tell. We can use boxplots to view outliers in more detail
# Boxplots, also called whisker charts, show how the values of a variable are distributed
# Boxplots include minimum, first quartile (Q1), median, third quartile (Q3), and maximum as well as outliers

# Boxplot of the 'Year Built'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily['Year Built'].dropna())
plt.title('Box Plot of Year Built')
plt.show()

# Boxplot of Floor Area (ft2)
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily['Self-Reported Gross Floor Area (ft²)'].dropna())
plt.title('Box Plot of Floor Area')
plt.show()

# Boxplot of Occupancy (%)
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily['Occupancy'].dropna())
plt.title('Box Plot of Occupancy')
plt.show()

# Boxplot of the 'Source EUI (kBtu/ft²)'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily['Source EUI (kBtu/ft²)'].dropna())
plt.title('Source EUI (kBtu/ft²)')
plt.show()


In [ ]:
# If we want to see our data overlaid on the boxplot, we can do that as well.
# This provides more information on how our data is distributed. Let's try this for Year Built.

plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily['Year Built'].dropna(), color='lightblue')

# Overlay the stripplot on the same axes to show individual data points
sns.stripplot(x=df_multifamily['Year Built'].dropna(), color='blue', alpha=0.5, jitter=True)

# Adding labels and title
plt.title('Boxplot with Underlying Data Points of Year Built for Multifamily Properties')
plt.xlabel('Year Built')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# From these visualizations, we can see floor area and source EUI both have some serious outliers
# Occupancy is almost all 100 percent, and so might not be that useful to us as as predictor
# Let's clean those up before we consider transformations and build our model
# We will start with floor area and try the method called interquartile range, which calculates the first and third quartiles
# and then determines the lower and upper bounds based on some factor of those quartiles. We will start with a factor of 1.5

# Calculate Q1 and Q3
Q1 = df_multifamily['Self-Reported Gross Floor Area (ft²)'].quantile(0.25)
Q3 = df_multifamily['Self-Reported Gross Floor Area (ft²)'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print('Q1 =',Q1, 'Q3 =', Q3, 'IQR =', IQR, 'lower bound =', lower_bound, 'upper bound =', upper_bound)

In [ ]:
# This IQR method isn't particularly helpful for floor area outliers since the lower bound came back negative
# We expect floor area and energy to be highly correlated, so let's visualize those two variables
# in a scatterplot and see if that's a better way to identify outliers

# First we create a new column 'Energy (kBtu)' by adding natural gas use and electricity use
df_multifamily['Energy (kBtu)'] = df_multifamily['Natural Gas Use (kBtu)'] + df_multifamily['Electricity Use - Grid Purchase (kBtu)']

# Scatter plot for Energy (kBtu) vs Self-Reported Gross Floor Area (ft²)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_multifamily, x='Self-Reported Gross Floor Area (ft²)', y='Energy (kBtu)')

plt.title('Energy vs Gross Floor Area')
plt.xlabel('Self-Reported Gross Floor Area (ft²)')
plt.ylabel('Energy (kBtu)')

plt.show()


In [ ]:
# This helps show where some of the real outliers are, which we can now try to remove more precisely
# We will try to do some minimal trimming of the Source EUI and just cut out values outside the 1st and 99th percentiles

# Calculate the 1st and 99th percentiles for 'Source EUI (kBtu/ft²)'
lower_percentile = df_multifamily['Source EUI (kBtu/ft²)'].quantile(0.01)
upper_percentile = df_multifamily['Source EUI (kBtu/ft²)'].quantile(0.99)

# Filter the dataFrame based on these percentiles and give it a new name (in case we need to try again)
df_multifamily_2 = df_multifamily[(df_multifamily['Source EUI (kBtu/ft²)'] > lower_percentile) & 
                                  (df_multifamily['Source EUI (kBtu/ft²)'] < upper_percentile)]

# Scatter plot for Energy (kBtu) vs Self-Reported Gross Floor Area (ft²)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_multifamily_2, x='Self-Reported Gross Floor Area (ft²)', y='Energy (kBtu)')

plt.title('Energy vs Gross Floor Area')
plt.xlabel('Self-Reported Gross Floor Area (ft²)')
plt.ylabel('Energy (kBtu)')

plt.show()

# Calculate and print the number of rows dropped
rows_dropped = df_multifamily.shape[0] - df_multifamily_2.shape[0]
print(f"Number of rows dropped: {rows_dropped}")


In [ ]:
# Let's look at our Source EUI again and see if it's looking better
# Boxplot of the 'Source EUI (kBtu/ft²)'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily_2['Source EUI (kBtu/ft²)'].dropna())
plt.title('Source EUI (kBtu/ft²)')
plt.show()

In [ ]:
# It's an improvement, but still a lot of outliers on the high side that we can try trimming now with the IQR method
# Calculate Q1 and Q3
Q1 = df_multifamily_2['Source EUI (kBtu/ft²)'].quantile(0.25)
Q3 = df_multifamily_2['Source EUI (kBtu/ft²)'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print('Q1 =',Q1, 'Q3 =', Q3, 'IQR =', IQR, 'lower bound =', lower_bound, 'upper bound =', upper_bound)

In [ ]:
# The lower bound from the IQR seems a bit aggressive, so let's just filter out outliers above the upper bound only
df_multifamily_2 = df_multifamily_2[df_multifamily_2['Source EUI (kBtu/ft²)'] <= upper_bound]

# Let's look at our Source EUI one more time and see if it's looking better
# Boxplot of the 'Source EUI (kBtu/ft²)'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily_2['Source EUI (kBtu/ft²)'].dropna())
plt.title('Source EUI (kBtu/ft²)')
plt.show()

In [ ]:
# That looks better for Source EUI and now floor area shows a clear correlation with energy
# Year built isn't the best variable to use as a predictor, so instead we'll transform it to Buiding Age

import datetime

# Calculate the current year
current_year = datetime.datetime.now().year

# Calculate building age for each building
df_multifamily_2.loc[:, 'Building Age'] = current_year - df_multifamily_2.loc[:, 'Year Built']

# Boxplot of Building Age
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_multifamily_2['Building Age'].dropna())
plt.title('Box Plot of Building Age')
plt.show()

In [ ]:
# Now that we have our data in better shape, we can use a pair plot (or scatterplot matrix) to visualize
# the relationships between our variables

data = df_multifamily_2[['Self-Reported Gross Floor Area (ft²)', 'Occupancy', 'Building Age', 'Source EUI (kBtu/ft²)']]

sns.pairplot(data)
plt.show()

In [ ]:
# We want to predict Source EUI (kBtu/ft²) using floor area, but floor area is an input to calculating Source EUI
# This creates a problem of circularity. So instead of using floor area directly, we will turn it into a
# category called Building Size Category and bin the floor area into three groups

import statsmodels.api as sm

# Bin the floor area and create categorical variable for three groups
bins = [0, 50000, 100000, float('inf')]
labels = ['Small', 'Medium', 'Large']
df_multifamily_2['Building Size Category'] = pd.cut(df_multifamily_2['Self-Reported Gross Floor Area (ft²)'], bins=bins, labels=labels)

# Get counts for each bin so we can see if they are roughly equal
bin_counts = df_multifamily_2['Building Size Category'].value_counts()
print("Counts per Building Size Category:")
print(bin_counts)

# One-hot encode the categorical variable, which turns our categories into 0s and 1s for medium and large buildings
# Small buildings do not get their own category since if we had three categories it would create issues of multicollinearity
# This is why we use the option drop_first=True (it drops the column for the first category)
df_multifamily_2 = pd.get_dummies(df_multifamily_2, columns=['Building Size Category'], drop_first=True)


In [ ]:
# Now we can prepare our model data and run a regression model using statsmodels
# We group our independent (or predictor) variables into a matrix called X and name our dependent variable y
X = df_multifamily_2[['Occupancy','Building Age','Building Size Category_Medium', 'Building Size Category_Large']]
y = df_multifamily_2['Source EUI (kBtu/ft²)']

# Add a constant to the model (it's required when using the statsmodels regression model)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print out the statistics
model.summary()

In [ ]:
# We see above that Occupancy was not statistically significant and also the note on condition number indicates we have
# multicollinearity in the model. So let's drop Occupancy and try again.

X = df_multifamily_2[['Building Age','Building Size Category_Medium', 'Building Size Category_Large']]
y = df_multifamily_2['Source EUI (kBtu/ft²)']

X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print out the statistics
model.summary()

In [ ]:
# We now have a valid model in that our coeffcients are significant and the condition number is much lower.
# The F-statistic, which determines if our model provides a better fit than a model with no parameters, is also significant.
# The Omnibus and Jarque-Bera tests are both significant, indicating the model's residuals are not normally distributed.
# The R2 is very low, and most of the model's predictive power is in the constant term, so it's not a great fit.
# This isn't surprising since we have limited predictor variables to use and we probably need more to get a better model.
# But we can now use our model to generate predictions of Source EUI

# First we create new column to generated our Predicted EUI
df_multifamily_2['Predicted Source EUI'] = model.predict(X)

# Next we create a column that divides actual Source EUI by predicted Source EUI just like the Energy Star methodology
df_multifamily_2['EUI Ratio'] = df_multifamily_2['Source EUI (kBtu/ft²)'] / df_multifamily_2['Predicted Source EUI']


In [ ]:
# Now we want to create a cumulative distribution of the EUI Ratio
# First we sort the EUI Ratio values
import numpy as np

sorted_ratio = np.sort(df_multifamily_2['EUI Ratio'])

# Then we calculate the cumulative distribution
cumulative_percent = np.arange(1, len(sorted_ratio)+1) / len(sorted_ratio)

# Create the plot to show the distribution
plt.figure(figsize=(10, 6))
plt.plot(sorted_ratio, cumulative_percent, marker='.', linestyle='none')
plt.title('Cumulative Distribution of Actual to Predicted Source EUI Ratio')
plt.xlabel('Actual/Predicted Source EUI Ratio')
plt.ylabel('Cumulative Percentage')

plt.grid(True)
plt.show()

In [ ]:
# Finally we can fit a gamma curve to the cumulative distribution just like Energy Star does
# If we were creating a real score, we would use the fitted gamme curve to calculate the score based on the
# Actual/Predicted Source EUI Ratio

from scipy.stats import gamma
from scipy.optimize import minimize
from scipy.integrate import cumtrapz

data = df_multifamily_2['EUI Ratio']

sorted_data = np.sort(data)
cumulative_data = np.arange(1, len(sorted_data)+1) / len(sorted_data)

# We can use basic statistical properties to inform initial guesses for curve shape and scale
mean_val = np.mean(data)
std_val = np.std(data)

initial_shape = mean_val**2 / std_val**2
initial_scale = std_val**2 / mean_val

# There are different methods to fit a gamma curve. We will use a negative log likelihood function
def neg_log_likelihood(params):
    a, loc, scale = params
    return -np.sum(gamma.logpdf(data, a, loc, scale))

# Optimize using maxiumum likelihood estimation (MLE), which helps find the parameters that make data most probable
# We use the initial guesses from above for shape and scale
initial_guess = [initial_shape, 0, initial_scale]
result = minimize(neg_log_likelihood, initial_guess, method='L-BFGS-B')

# Extract the best fit parameters
a, loc, scale = result.x

# Calculate x values
x_values = np.linspace(sorted_data.min(), sorted_data.max(), 1000)

# Calculate normalized CDF values
pdf_values = gamma.pdf(x_values, a, loc, scale)
cdf_values = cumtrapz(pdf_values, x_values, initial=0)
cdf_values /= cdf_values[-1]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(sorted_data, cumulative_data, label='Empirical Cumulative Distribution', marker='.', linestyle='none')
plt.plot(x_values, cdf_values, label='Fitted Gamma Distribution', color='red')
plt.title('Cumulative Distribution and Fitted Gamma Curve')
plt.xlabel('Actual/Predicted Source EUI Ratio')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.grid(True)
plt.show()

# Print the best fit parameters
print(f"Fitted Gamma Distribution Parameters:\n Shape (a): {a}\n Location (loc): {loc}\n Scale: {scale}")
